In [44]:
import tensorflow as tf
import numpy as np
import os
from matplotlib import pyplot as plt
from tensorflow import keras
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix,plot_confusion_matrix

from tensorflow.keras.utils import to_categorical  
import tensorflow.keras.utils as util
from tensorflow.keras.layers.experimental import preprocessing
import ast
import os

In [34]:
import splitfolders

In [35]:
from google.colab import drive
drive.mount('/content/drive')

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [36]:
def mean_of_geometric(data):
    F_lower=[[],[]]
    F=[]
    d=[]
    f=[]

    ox=np.array([1,0,0]).reshape(-1,1)
    oy=np.array([0,1,0]).reshape(-1,1)
    oz=np.array([0,0,1]).reshape(-1,1)

    for N in range(1, (len(data)//20)+1):
      F_lower=[[],[]]
      for i in range((N-1)*20, N*20):
        for j in range(i+1, N*20):
          (xi, yi, zi)=data[i]
          (xj, yj, zj)=data[j]
          ji=np.sqrt((yj-yi)**2+(zj-zi)**2+(xj-xi)**2)

          d.append(np.sqrt((yj-yi)**2+(zj-zi)**2))
          d.append(np.sqrt((xj-xi)**2+(zj-zi)**2))
          d.append(np.sqrt((xj-xi)**2+(yj-yi)**2))
          F_lower[0].append(d)

          if ji!=0.0:
            t1=np.arccos(np.clip((np.dot(np.array(d), oy)//ji), -1.0, 1.0))
            f.append(t1[0])
            t2=np.arccos(np.clip((np.dot(np.array(d), oz)//ji), -1.0, 1.0))
            f.append(t2[0])
            t3=np.arccos(np.clip((np.dot(np.array(d), ox)//ji), -1.0, 1.0))
            f.append(t3[0])
            F_lower[1].append(f)
          else:
            f.append(0.0)
            f.append(0.0)
            f.append(0.0)
            F_lower[1].append(f)

          d=[]
          f=[]
      F.append(F_lower)
      
    F_means=[]
    F_mean=[[], []]
    
    m=[[],[]]
    tem=np.zeros((190, 3)).tolist()
    m[0]=tem
    m[1]=tem

    count=0
    k=0
    while k<len(F):
        if count==30:
            k=k-24
            count=0
            m=(np.array(m)/30).tolist()
            F_mean[0]=m[0]
            F_mean[1]=m[1]
            
            F_means.append(F_mean)
            
            
            F_mean=[[], []]
            
            m=[[],[]]
            tem=np.zeros((190, 3)).tolist()
            m[0]=tem
            m[1]=tem
            
        for j in range(0, 190): 
          m[0][j]=(np.array(m[0][j])+np.array(F[k][0][j])).tolist()
          m[1][j]=np.array(m[1][j])+np.array(F[k][1][j]).tolist()
        
        
        if k==len(F)-1 and count!=30 and count!=0:
            t=k-(k//30)*30
            if t!=0:
                m=(np.array(m)/t).tolist()
                F_mean[0]=m[0]
                F_mean[1]=m[1]
                
                F_means.append(F_mean)
                
        count+=1
        k+=1

    return F_means

(150,)

In [37]:
path = "/content/drive/My Drive/GaitSequences1/test"
subfolders = [ f.path for f in os.scandir(path) if f.is_dir() ]

data_per_sequence=[]
test_ds=[[],[]]
        
for directory in subfolders:
    label=int(os.path.basename(directory).lstrip('Person'))
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        if os.path.isfile(f):
            if f.endswith(".txt"):
                data_per_sequence=[]
                with open(f, 'r') as fl:
                    var=fl.read()
                    for line in var.splitlines():
                        aList = list(ast.literal_eval(line))
                        data_per_sequence.append(aList)
                    f_means=mean_of_geometric(data_per_sequence)
                    for i in range(0, len(f_means)):
                        test_ds[0].append(f_means[i])
                        test_ds[1].append((label-1))

In [38]:
print(len(test_ds[0]))
print(len(test_ds[1]))
print(np.array(test_ds[0]).shape)
print(np.array(test_ds[1]).shape)
print(test_ds[1])

In [39]:
from sklearn.utils import shuffle
train_data, train_label = shuffle(train_ds[0], train_ds[1])

nsamples, nx, ny, nz = np.array(train_data).shape
train_dataset = np.array(train_data).reshape((nsamples,nx*ny*nz))
nsamples, nx, ny, nz = np.array(test_ds[0]).shape
test_dataset = np.array(test_ds[0]).reshape((nsamples,nx*ny*nz))

print(train_dataset.shape)
print(np.array(train_ds[1]).shape)
print(test_dataset.shape)
print(np.array(test_ds[1]).shape)

from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier(n_estimators=100)
clf.fit(train_dataset, train_label)
y_pred = clf.predict(test_dataset)
print("RF Accuracy: %.3f" %metrics.accuracy_score(test_ds[1], y_pred))


RandomForestClassifier()

In [42]:
from joblib import dump

In [43]:
dump(model, './../savedModels/model.joblib')

['./../savedModels/model.joblib']